In [1]:
import numpy as np
import pandas as pd
import torch
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import seaborn as sns
from tqdm import tqdm
from torch import nn
from sklearn.metrics import accuracy_score
from umap import UMAP

/home/boris/Documents/kaggle cassava/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
%matplotlib inline

/home/boris/Documents/kaggle cassava/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
cv_results = context.catalog.load('cv_results')

2020-12-09 17:18:26,289 - kedro.io.data_catalog - INFO - Loading data from `cv_results` (PickleDataSet)...


In [11]:
print(f"Accuracy: {cv_results['accuracy_mean']:0.4f} +- {cv_results['accuracy_std']:0.4f}")

Accuracy: 0.5909 +- 0.0218


In [12]:
print(f"F1-score: {cv_results['f1_score_mean']:0.4f} +- {cv_results['f1_score_std']:0.4f}")

F1-score: 0.6549 +- 0.0411
